In [9]:
import argparse
import os
import shutil
import time
import os, sys, pdb, shutil, time, random, datetime

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from utils import convert_secs2time, time_string, time_file_str,AverageMeter
import torch.optim.lr_scheduler as lr_scheduler
# from models import print_log
import models
from tensorboardX import SummaryWriter
from utils import convert_model, measure_model



model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
args = parser.parse_args()
args.prefix = time_file_str()

best_prec1 = 0
best_prec5 = 0

writer = SummaryWriter()

if not os.path.isdir(args.save_dir):
  os.makedirs(args.save_dir)
# used for file names, etc 
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')     
log = open(os.path.join(args.save_dir, '{}.{}_{}.log'.format(args.arch, args.prefix, time_stamp)), 'w')

# create model
print_log("=> creating model '{}'".format(args.arch), log)
model = models.__dict__[args.arch](1000)
print_log("=> Model : {}".format(model), log)
print_log("=> parameter : {}".format(args), log)

if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
  model.features = torch.nn.DataParallel(model.features)
  model.cuda()
else:
  model = torch.nn.DataParallel(model).cuda()

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()


# optimizer = torch.optim.Adadelta(model.parameters(), weight_decay=args.weight_decay,
#                                  lr=0.1, rho=0.9)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
            momentum=args.momentum,
            weight_decay=args.weight_decay,
            nesterov=True)


IMAGE_SIZE=224
print_log(summary(model, input_size=(3, IMAGE_SIZE, IMAGE_SIZE)), log)


n_flops, n_params = measure_model(model, IMAGE_SIZE, IMAGE_SIZE)
print_log('FLOPs: %.2fM, Params: %.2fM' % (n_flops / 1e6, n_params / 1e6), log)

## epoch 
milestones = [30, 60, 90, 130, 150]#[10, 20, 30, 40, 50, 60]#[15, 30, 60, 90, 110, 140]
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1)

print(str(scheduler))
temp = scheduler.state_dict()
temp['last_epoch']=36
raw_input()
#scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print_log("=> loading checkpoint '{}'".format(args.resume), log)
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_prec1 = checkpoint['best_prec1']
        if 'best_prec5' in checkpoint:
            best_prec5 = checkpoint['best_prec5']
        else:
            best_prec5 = 0.00
        model.load_state_dict(checkpoint['state_dict'])
        if 1==1:
            scheduler.load_state_dict(temp)
           #scheduler.load_state_dict(checkpoint['scheduler'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        model.eval()
        print_log("=> loaded checkpoint '{}' (epoch {})".format(args.resume, checkpoint['epoch']), log)
    else:
        print_log("=> no checkpoint found at '{}'".format(args.resume), log)

cudnn.benchmark = True

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-aa2fd69e-fcbf-4a8a-b6e4-cef55193e856.json


SystemExit: 2

/home/hossein/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
